# Scrapping

In [38]:
from bs4 import BeautifulSoup
import requests as req
from splinter import Browser
from IPython.display import display_html
import pandas as pd

In [39]:
# Showing the chromdriver I'm using
!which chromedriver

/usr/local/bin/chromedriver


In [40]:
# Executable path and fire up browser, headless set to True so the browser operates in the background invisible 
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

#### NASA Mars News

In [41]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [42]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [43]:
# Find title and place it into a variable
news_title = soup.find('div', class_="content_title").text
# Print for verification
print(news_title)

Want to Colonize Mars? Aerogel Could Help


In [44]:
# Find paragraph text and put it into a variable
news_p = soup.find('div', class_="article_teaser_body").text
# Print for verification
print(news_p)

Researchers are studying whether a wonder material used in Mars rovers could help warm parts of the Red Planet rich in water ice.


#### JPL Mars Space Images - Featured Image

In [45]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [46]:
#Use splinter to navigate the site and find the image url for the current Featured Mars Image
browser.click_link_by_partial_text('FULL IMAGE')

In [47]:
browser.click_link_by_partial_text('more info')

In [48]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [49]:
# Assign the url string to a variable called featured_image_url.
image = soup.find('img', class_= "main_image")
featured_image_url = "https://www.jpl.nasa.gov" + image["src"]

#### Mars Weather

In [50]:
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)

In [51]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [52]:
tweets = soup.find("p", class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text
print(tweets)

InSight sol 222 (2019-07-12) low -99.7ºC (-147.5ºF) high -24.8ºC (-12.6ºF)
winds from the SSE at 4.2 m/s (9.4 mph) gusting to 15.6 m/s (34.8 mph)
pressure at 7.60 hPapic.twitter.com/8Q8lyB6SjM


#### Mars Facts

In [53]:
# Set url
url = 'https://space-facts.com/mars/'
# Use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
table = pd.read_html(url)

In [54]:
# Use Pandas to convert the data to a HTML table string. 
table_html_string = table[1].to_html()
# Print the string to see I have retrieved what I want
print (table_html_string)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>0</th>
      <th>1</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
      <td>Recorded

#### Mars Hemispheres

In [55]:
# Go to url
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [56]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [15]:
hemi_attributes_list = soup.find_all('a', class_="itemLink product-item")
print(hemi_attributes_list[1]['href'])

/search/map/Mars/Viking/cerberus_enhanced


In [27]:
hemisphere_image_urls = []
for hemi_img in hemi_attributes_list:
    if hemi_img.find('h3') == None:
        pass
    else:
        img_title = hemi_img.find('h3').text
        #print(img_title)
        link_to_img = "https://astrogeology.usgs.gov/" + hemi_img['href']
        #print(link_to_img)
        img_request = req.get(link_to_img)
        soup = BeautifulSoup(img_request.text, 'lxml')
        img_tag = soup.find('div', class_='downloads')
        img_url = img_tag.find('a')['href']
        hemisphere_image_urls.append({"title": img_title, "img_url": img_url})

In [28]:
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
